**setup and imports**

In [2]:
!pip install tensorflow hmm numpy hmmlearn
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from hmmlearn import hmm
import random
import string
import re
from collections import defaultdict, deque
import os
import time

# Suppress TensorFlow warnings for cleaner output
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
tf.get_logger().setLevel('ERROR')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 kB 7.9 MB/s eta 0:00:00


**Loading and Preprocessing**


In [3]:
def load_corpus(filename="corpus.txt"):
    """
    Loads the 50,000-word corpus[cite: 18, 37].
    The provided file  is a single text block.
    """
    print(f"Loading corpus from {filename}...")
    try:
        # Use the content from the provided 'corpus.txt'
        with open(filename, 'r') as f:
            full_content = f.read()

        # Split by any whitespace and filter for valid words
        all_words = re.split(r'\s+', full_content)
        valid_words = sorted(list(set(
            word.lower() for word in all_words if word.isalpha()
        )))

        print(f"Loaded {len(valid_words)} unique, valid words.")
        return valid_words

    except FileNotFoundError:
        print(f"Error: {filename} not found.")
        print("Please ensure 'corpus.txt' is in the same directory.")
        return []

def group_words_by_length(words):
    """
    Groups words by their length, as hinted for HMM training.
    """
    words_by_length = defaultdict(list)
    for word in words:
        words_by_length[len(word)].append(word)
    print(f"Grouped words into {len(words_by_length)} length-based buckets.")
    return words_by_length

def word_to_sequence(word):
    """Converts a word into a numpy array of integer emissions (0-25)."""
    return np.array([ord(char) - ord('a') for char in word]).reshape(-1, 1)

def sequence_to_word(seq):
    """Converts a sequence of integers back to a word."""
    return "".join([chr(ord('a') + num) for num in seq.flatten()])

**HMM**

In [4]:
class HMMOracle:
    """
    Implements the hybrid probabilistic oracle[cite: 19].
    It uses a word-list filter as the primary source and a
    positional HMM as a fallback.
    """
    def __init__(self, words_by_length):
        self.words_by_length = words_by_length
        self.max_len = max(words_by_length.keys())
        self.hmms = {} # Stores HMMs by length

        # Build regex patterns for word filtering
        self.pattern_cache = {}

    def _get_pattern(self, masked_word, guessed_letters):
        """Creates a regex pattern to filter the word list."""
        key = (masked_word, tuple(sorted(guessed_letters)))
        if key in self.pattern_cache:
            return self.pattern_cache[key]

        pattern = list(masked_word)
        available_letters = set(string.ascii_lowercase) - guessed_letters

        # Letters that are KNOWN to not be in the word
        wrong_guesses = guessed_letters - set(masked_word)

        # Regex:
        # ^...$ : full word match
        # [^abc]: blank spot cannot be a wrong guess
        # p: known letter must be 'p'

        regex_parts = []
        for char in pattern:
            if char == '_':
                # Blank must be a letter that hasn't been guessed wrong
                negation_set = "".join(sorted(wrong_guesses))
                # Fix: Handle empty negation set
                if negation_set:
                    regex_parts.append(f"[^{negation_set}]")
                else:
                    regex_parts.append(".") # Any character if no wrong guesses
            else:
                # Must be this exact letter
                regex_parts.append(char)

        regex_str = f"^{''.join(regex_parts)}$"
        self.pattern_cache[key] = re.compile(regex_str)
        return self.pattern_cache[key]

    def train(self):
        """
        Trains one HMM for each word length[cite: 18, 62].
        State = Position in word
        Emission = Letter
        """
        print("Training HMMs (positional frequency models)...")
        for length, word_list in self.words_by_length.items():
            if not word_list:
                continue

            # This is a degenerate HMM where states are positions.
            # We fix the transitions to be strictly left-to-right.
            model = hmm.CategoricalHMM(
                n_components=length, # n_states = word length
                n_features=26,       # n_emissions = 'a'-'z'
                init_params="",      # We will set all params
                params=""            # Don't train anything
            )

            model.startprob_ = np.array([1.0] + [0.0] * (length - 1))

            transmat = np.zeros((length, length))
            for i in range(length - 1):
                transmat[i, i + 1] = 1.0
            transmat[length - 1, length - 1] = 1.0 # End state loops
            model.transmat_ = transmat

            # Calculate emission probabilities (P(letter | position))
            emission_prob = np.ones((length, 26)) # Add-1 smoothing
            for word in word_list:
                for i, char in enumerate(word):
                    emission_prob[i, ord(char) - ord('a')] += 1

            # Normalize
            emission_prob /= np.sum(emission_prob, axis=1, keepdims=True)
            model.emissionprob_ = emission_prob

            self.hmms[length] = model
        print("HMM training complete.")

    def get_probabilities(self, masked_word, guessed_letters):
        """
        Calculates the probability distribution over the alphabet[cite: 24].

        This is the core "oracle" function [cite: 19] that estimates
        P(letter | masked_word, guessed_letters).
        """
        length = len(masked_word)
        if length not in self.words_by_length:
            return np.zeros(26) # No words of this length

        # 1. Primary Strategy: Word List Filtering
        # Find all words that match the current state
        pattern = self._get_pattern(masked_word, guessed_letters)
        relevant_words = [
            word for word in self.words_by_length[length] if pattern.match(word)
        ]

        probs = np.zeros(26)
        available_letters = set(string.ascii_lowercase) - guessed_letters

        if relevant_words:
            # We found matches. Calculate freqs from this relevant subset.
            total_matches = len(relevant_words)
            for word in relevant_words:
                # Only count letters that are in blank spots
                for i, char in enumerate(word):
                    if masked_word[i] == '_':
                        probs[ord(char) - ord('a')] += 1

            # Normalize and mask out already-guessed letters
            if np.sum(probs) > 0:
                probs /= np.sum(probs)

            for i in range(26):
                if chr(ord('a') + i) not in available_letters:
                    probs[i] = 0

            return probs

        # 2. Fallback Strategy: Use the HMM
        # No relevant words were found (e.g., start of game or rare pattern)
        # Use the positional HMM probabilities for the blank spots.
        model = self.hmms.get(length)
        if model is None:
            return np.zeros(26) # Should not happen if trained

        probs = np.zeros(26)
        for i, char in enumerate(masked_word):
            if char == '_':
                # Add the probability of all letters at this position
                probs += model.emissionprob_[i, :]

        # Mask out already-guessed letters
        for i in range(26):
            if chr(ord('a') + i) not in available_letters:
                probs[i] = 0

        # Normalize
        total_prob = np.sum(probs)
        if total_prob > 0:
            return probs / total_prob
        else:
            # No possible letters left (should be a game-over state)
            return np.zeros(26)

Reinforcement Learning

In [5]:
class HangmanEnv:
    """Implements the Hangman Game Environment for the RL agent."""

    def __init__(self, word_list, oracle, max_lives=6): # 6 wrong guesses allowed
        self.all_words = word_list
        self.oracle = oracle
        self.max_lives = max_lives
        self._reset_game_stats()

    def _reset_game_stats(self):
        """Generates a new game state."""
        # Words are drawn from the provided corpus only
        self.word = random.choice(self.all_words)
        self.word_len = len(self.word)
        self.masked_word = "_" * self.word_len
        self.lives_left = self.max_lives
        self.guessed_letters = set()
        self.game_wrong_guesses = 0
        self.game_repeated_guesses = 0
        self.game_won = False

    def _get_state(self):
        """
        Defines the state representation for the RL agent.
        State is a fixed-size vector:
        - [0:26]: HMM probability distribution
        - [26:52]: Binary vector of guessed letters
        - [52]: Normalized number of lives left
        - [53]: Normalized number of blank spots
        """
        # Get probability distribution from the HMM Oracle
        hmm_probs = self.oracle.get_probabilities(
            self.masked_word, self.guessed_letters
        )

        # Create guessed letters vector
        guessed_vec = np.array([
            1.0 if c in self.guessed_letters else 0.0
            for c in string.ascii_lowercase
        ])

        # Create metadata
        lives_norm = self.lives_left / self.max_lives
        blanks_norm = self.masked_word.count('_') / self.word_len

        # Concatenate into a single state vector
        state = np.concatenate([
            hmm_probs,
            guessed_vec,
            [lives_norm],
            [blanks_norm]
        ])
        return state.astype(np.float32)

    def reset(self):
        """Resets the environment and returns the initial state."""
        self._reset_game_stats()
        return self._get_state()

    def step(self, action):
        """
        Executes one action (guessing a letter) in the environment.
        Action: int 0-25, corresponding to 'a'-'z'.
        """
        letter = chr(ord('a') + action)

        done = False
        info = {
            "repeated": 0,
            "wrong": 0,
            "won": False,
            "word": self.word,
            "masked": self.masked_word
        }

        # 1. Check for Repeated Guess
        if letter in self.guessed_letters:
            self.game_repeated_guesses += 1
            info["repeated"] = 1
            # Reward defined by the scoring formula
            reward = -2
            return self._get_state(), reward, done, info

        self.guessed_letters.add(letter)

        # 2. Check for Correct/Wrong Guess
        if letter in self.word:
            # Correct guess
            new_masked_word = list(self.masked_word)
            letters_found = 0
            for i, char in enumerate(self.word):
                if char == letter:
                    new_masked_word[i] = letter
                    letters_found += 1
            self.masked_word = "".join(new_masked_word)

            # --- REWARD CHANGE ---
            # Simpler, more direct reward for progress
            reward = 10
            info["wrong"] = 0

        else:
            # Wrong guess
            self.lives_left -= 1
            info["wrong"] = 1
            self.game_wrong_guesses += 1
            # Reward defined by the scoring formula
            reward = -5

        # 3. Check for Game Over (Win or Lose)
        if "_" not in self.masked_word:
            # Game Won
            done = True
            self.game_won = True
            info["won"] = True
            # --- REWARD CHANGE ---
            # Scaled down to be more balanced with penalties
            reward = 100

        elif self.lives_left == 0:
            # Game Lost
            done = True
            self.game_won = False
            info["won"] = False
            # --- REWARD CHANGE ---
            # Added a large, explicit penalty for losing
            reward = -100

        return self._get_state(), reward, done, info

**RL Agent**

In [6]:
class DQNAgent:
    """
    Implements a Deep Q-Network (DQN) agent.
    """
    def __init__(self, state_size, action_size, learning_rate=0.001, gamma=0.99): #
        self.state_size = state_size   # 54
        self.action_size = action_size # 26

        # Experience Replay Buffer
        self.memory = deque(maxlen=100000)

        # Hyperparameters
        self.gamma = gamma    # Discount factor (increased for long-term planning)

        # --- TUNING CHANGE 1 ---
        # Lowered learning rate for more stable Q-value convergence
        self.learning_rate = learning_rate

        # Exploration-Exploitation Strategy
        self.epsilon = 1.0
        self.epsilon_min = 0.01

        # --- TUNING CHANGE 2 ---
        # Faster decay. We want the agent to stop guessing randomly sooner.
        self.epsilon_decay = 0.9992

        # Q-Network and Target Network
        self.model = self._build_model()
        self.target_model = self._build_model()
        self.update_target_model()

    def _build_model(self):
        """Builds the neural network to approximate the Q-function."""
        model = Sequential([
            Input(shape=(self.state_size,)),
            Dense(128, activation='relu'),
            Dense(128, activation='relu'),
            Dense(64, activation='relu'),
            Dense(self.action_size, activation='linear') # Q-values for each action
        ])
        model.compile(
            loss='mse',
            optimizer=Adam(learning_rate=self.learning_rate)
        )
        return model

    def update_target_model(self):
        """Syncs the Target Network with the weights of the Main Network."""
        self.target_model.set_weights(self.model.get_weights())

    def remember(self, state, action, reward, next_state, done):
        """Stores an experience tuple in the replay buffer."""
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        """
        Chooses an action using an epsilon-greedy policy.
        It *must* mask out (avoid) actions that have already been guessed.
        """
        # Extract the guessed_vec from the state (indices 26 to 52)
        guessed_mask = state[0][26:52].astype(bool)

        # Get all actions that are *not* guessed
        available_actions = [i for i, guessed in enumerate(guessed_mask) if not guessed]

        if not available_actions:
            # Fallback in case no actions are available
            return 0

        # Exploration
        if np.random.rand() <= self.epsilon:
            return random.choice(available_actions)

        # Exploitation
        act_values = self.model.predict(state, verbose=0)[0]

        # Mask out unavailable actions by setting their Q-value to -infinity
        masked_act_values = [
            act_values[i] if i in available_actions else -np.inf
            for i in range(self.action_size)
        ]

        return np.argmax(masked_act_values)

    def replay(self, batch_size):
        """Trains the model on a minibatch from the replay buffer."""
        if len(self.memory) < batch_size:
            return # Not enough samples yet

        minibatch = random.sample(self.memory, batch_size)

        states = np.vstack([t[0] for t in minibatch])
        next_states = np.vstack([t[3] for t in minibatch])

        # Predict Q-values for current states and next states
        q_values_current = self.model.predict(states, verbose=0)
        q_values_next_target = self.target_model.predict(next_states, verbose=0)

        targets = []
        for i, (state, action, reward, next_state, done) in enumerate(minibatch):
            if done:
                target = reward
            else:
                # Bellman equation: Q(s,a) = r + gamma * max_a'(Q_target(s',a'))
                target = reward + self.gamma * np.amax(q_values_next_target[i])

            # Get the Q-values for the batch's i-th state
            current_q_target = q_values_current[i]
            # Update only the Q-value for the action that was taken
            current_q_target[action] = target
            targets.append(current_q_target)

        # Train the model on the (states, targets) batch
        self.model.fit(states, np.array(targets), epochs=1, verbose=0)

        # Decay epsilon
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

**Training loop**

In [7]:
def main():
    # --- 1. Load and Prepare Data ---
    all_words = load_corpus("corpus.txt")
    if not all_words:
        return

    words_by_length = group_words_by_length(all_words)

    # --- 2. Build HMM Oracle ---
    oracle = HMMOracle(words_by_length)
    oracle.train() # Train on the corpus

    # --- 3. Build Environment and Agent ---
    env = HangmanEnv(all_words, oracle, max_lives=6)

    STATE_SIZE = 54 # 26 (HMM) + 26 (Guessed) + 1 (Lives) + 1 (Blanks)
    ACTION_SIZE = 26 # 'a' - 'z'

    # Agent will now use the new hyperparameters
    agent = DQNAgent(STATE_SIZE, ACTION_SIZE)

    # --- 4. Training Loop ---
    # --- TUNING CHANGE 3 ---
    # More episodes to allow the smaller learning rate to converge
    EPISODES = 6500
    BATCH_SIZE = 64
    # --- TUNING CHANGE 4 ---
    # Update target network *much* less frequently for stability
    UPDATE_TARGET_FREQ = 500

    print(f"\n--- Starting RL Agent Training ({EPISODES} episodes) ---")
    start_time = time.time()
    episode_rewards = []

    for e in range(1, EPISODES + 1):
        state = env.reset()
        state = np.reshape(state, [1, STATE_SIZE])

        done = False
        total_episode_reward = 0

        while not done:
            action = agent.act(state)
            next_state, reward, done, info = env.step(action)
            next_state = np.reshape(next_state, [1, STATE_SIZE])

            agent.remember(state, action, reward, next_state, done)

            state = next_state
            total_episode_reward += reward

            if done:
                episode_rewards.append(total_episode_reward)

        # Train the agent from replay buffer
        agent.replay(BATCH_SIZE)

        # Update target network
        if e % UPDATE_TARGET_FREQ == 0:
            agent.update_target_model()
            print(f"--- Target network updated at episode {e} ---") # Log this

        if e % 100 == 0:
            avg_reward = np.mean(episode_rewards[-100:])
            print(f"Episode: {e}/{EPISODES} | "
                  f"Avg Reward (last 100): {avg_reward:.2f} | "
                  f"Epsilon: {agent.epsilon:.3f}")

    training_time = time.time() - start_time
    print(f"--- Training Finished in {training_time:.2f}s ---")

    # --- 5. Evaluation ---
    EVAL_GAMES = 2000
    print(f"\n--- Starting Evaluation ({EVAL_GAMES} games) ---")

    # Set agent to exploitation-only mode
    agent.epsilon = 0.0

    total_wins = 0
    total_wrong_guesses = 0
    total_repeated_guesses = 0

    # Load the hidden test set
    try:
        with open("test.txt", 'r') as f:
            test_content = f.read()
        test_words = re.split(r'\s+', test_content)
        test_words = [word.lower() for word in test_words if word.isalpha()]
        env.all_words = test_words # Force env to use test words
        print(f"Loaded {len(test_words)} words from test.txt for evaluation.")
    except FileNotFoundError:
        print("Warning: test.txt not found. Evaluating on the training corpus.")
        pass


    for g in range(1, EVAL_GAMES + 1):
        state = env.reset()
        state = np.reshape(state, [1, STATE_SIZE])

        done = False
        game_wrong = 0
        game_repeated = 0

        while not done:
            action = agent.act(state)
            next_state, reward, done, info = env.step(action)
            state = np.reshape(next_state, [1, STATE_SIZE])

            game_wrong += info["wrong"]
            game_repeated += info["repeated"]

            if done:
                if info["won"]:
                    total_wins += 1
                total_wrong_guesses += game_wrong
                total_repeated_guesses += game_repeated

        if g % 200 == 0:
            print(f"Played game {g}/{EVAL_GAMES}...")

    # --- 6. Final Results ---
    print("\n--- 🏁 Final Evaluation Results ---")

    success_rate = total_wins / EVAL_GAMES
    avg_wrong = total_wrong_guesses / EVAL_GAMES
    avg_repeated = total_repeated_guesses / EVAL_GAMES

    # Calculate final score based on the formula
    final_score = (success_rate * 2000) - (total_wrong_guesses * 5) - (total_repeated_guesses * 2)

    print(f"Total Games Played: {EVAL_GAMES}")
    print("\n--- Averages ---")
    print(f"Success Rate:         {success_rate * 100:.2f}%")
    print(f"Avg. Wrong Guesses:   {avg_wrong:.3f}")
    print(f"Avg. Repeated Guesses: {avg_repeated:.3f}")

    print("\n--- Totals ---")
    print(f"Total Wins:             {total_wins}")
    print(f"Total Wrong Guesses:    {total_wrong_guesses}")
    print(f"Total Repeated Guesses: {total_repeated_guesses}")

    print("\n--- SCORE ---")
    print(f"Success Points:  ( {success_rate:.3f} * 2000 )   = {success_rate * 2000:,.0f}")
    print(f"Wrong Penalty:   ( {total_wrong_guesses} * 5 )      = -{total_wrong_guesses * 5:,.0f}")
    print(f"Repeat Penalty:  ( {total_repeated_guesses} * 2 )      = -{total_repeated_guesses * 2:,.0f}")
    print(f"**Final Score**:                            = **{final_score:,.0f}**")


if __name__ == "__main__":
    main()

Loading corpus from corpus.txt...
Loaded 49399 unique, valid words.
Grouped words into 24 length-based buckets.
Training HMMs (positional frequency models)...
HMM training complete.

--- Starting RL Agent Training (20000 episodes) ---
Episode: 100/20000 | Avg Reward (last 100): -100.40 | Epsilon: 0.928
Episode: 200/20000 | Avg Reward (last 100): -96.80 | Epsilon: 0.856
Episode: 300/20000 | Avg Reward (last 100): -92.10 | Epsilon: 0.790
Episode: 400/20000 | Avg Reward (last 100): -95.10 | Epsilon: 0.730
--- Target network updated at episode 500 ---
Episode: 500/20000 | Avg Reward (last 100): -89.90 | Epsilon: 0.673
Episode: 600/20000 | Avg Reward (last 100): -87.55 | Epsilon: 0.622
Episode: 700/20000 | Avg Reward (last 100): -86.10 | Epsilon: 0.574
Episode: 800/20000 | Avg Reward (last 100): -87.20 | Epsilon: 0.530
Episode: 900/20000 | Avg Reward (last 100): -81.55 | Epsilon: 0.489
--- Target network updated at episode 1000 ---
Episode: 1000/20000 | Avg Reward (last 100): -78.40 | Epsil

KeyboardInterrupt: 